In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import sqrt, floor, log10, cos, sin


def round_sig(x: float, sig: int) -> float:
    """Rounds x to sig significant digits"""
    return x if x == 0 else round(x, sig - int(floor(log10(abs(x)))) - 1)


plt.rcParams["figure.figsize"] = (20, 28)
# plt.style.use('dark_background')


### Ładowanie danych

In [ ]:
df = pd.read_csv("data.csv")
df.head()


### Wykres I($\phi$)

In [ ]:
# 1 degree = pi / 180 radians

I = df["I"]
phi_radians = df["fi"] * np.pi / 180  # in radians
phi_error_radians = [0.5 * np.pi / 180 for _ in range(len(phi_radians))]  # in radians
I_error = [round_sig((8 / 1000 * i + 2 * 0.1) / sqrt(3), 2) for i in I]

# set polar coordinates
ax = plt.axes(polar=True)
ax.set_theta_zero_location("N")
ax.plot(phi_radians, I, color="red")
ax.errorbar(
    phi_radians,
    I,
    yerr=I_error,
    xerr=phi_error_radians,
    fmt="o",
    linewidth=1,
    markersize=2,
    color="red",
)

# change size of x and y axis numbers
ax.tick_params(axis="both", which="major", labelsize=15)

# set x axis every 5 degrees
ax.set_xticks(np.arange(0, 2 * np.pi, np.pi / 36))

plt.title(r"Wykres wskazań miernika w zależności od kąta $I(\phi)$", fontsize=30)
plt.savefig("./img/wykres1.jpg")
plt.show()


### Obliczenie teoretycznych wartości prądu 
#### $I_T = I_{max} \cdot cos^2(\phi)$

In [ ]:
I_max = max(df["I"])
I_T = [round_sig(I_max * cos(phi) ** 2, 3) for phi in phi_radians]

# 1 degree = pi / 180 radians

I_T_error = [
    round_sig(
        sqrt(
            (2 * I_max * cos(phi) * sin(phi) * phi_error) ** 2
            + (i_error * cos(phi) ** 2) ** 2
        ),
        2,
    )
    for phi, phi_error, i_error in zip(phi_radians, phi_error_radians, I_error)
]


# set polar coordinates
ax = plt.axes(polar=True)
ax.set_theta_zero_location("N")
ax.errorbar(
    phi_radians,
    I_T,
    yerr=I_T_error,
    xerr=phi_error_radians,
    fmt="o",
    linewidth=1,
    markersize=2,
    color="midnightblue",
)

# change size of x and y axis numbers
ax.tick_params(axis="both", which="major", labelsize=15)

# set x axis every 5 degrees
ax.set_xticks(np.arange(0, 2 * np.pi, np.pi / 36))

plt.title(r"Wykres wartości teoretycznych $I_T(\phi)$", fontsize=30)
plt.savefig("./img/wykres2.jpg")
plt.show()


### Obliczenie błąd zera $\alpha$
### $\alpha = \phi_T - \phi$, $\alpha = 110^{\circ}$

In [ ]:
alpha = 110 * np.pi / 180
rotated_phi = [alpha + phi for phi in phi_radians]

# set polar coordinates
ax = plt.axes(polar=True)
ax.set_theta_zero_location("N")
ax.plot(phi_radians, I, color="red")
ax.errorbar(
    phi_radians,
    I,
    yerr=I_error,
    xerr=phi_error_radians,
    fmt="o",
    linewidth=1,
    markersize=2,
    label=r"$I(\phi)$, $\mu$A",
    color="red",
)

ax.plot(rotated_phi, I_T, color="midnightblue")
ax.errorbar(
    rotated_phi,
    I_T,
    yerr=I_T_error,
    xerr=phi_error_radians,
    fmt="o",
    linewidth=1,
    markersize=2,
    label=r"$I_T(\phi+\alpha)$, $\mu$A",
    color="midnightblue",
)

# change size of x and y axis numbers
ax.tick_params(axis="both", which="major", labelsize=15)

# set legend
ax.legend(loc="upper left", fontsize=20)

# set x axis every 5 degrees
ax.set_xticks(np.arange(0, 2 * np.pi, np.pi / 36))

plt.title(r"Nałożone wykresy, kąt obrotu $\alpha=110^{\circ}$", fontsize=30)
plt.savefig("./img/wykres3.jpg")
plt.show()
